In [1]:
%matplotlib inline
%config InlineBackend.figure_format = "retina"
from psfutil_1d import *
from matplotlib.colors import LogNorm

# 2. An Essay on PSF

In [2]:
# 2025.09.07 regrid_1d.ipynb
BAND = "H158"

psf_in = psf_single_slit(LDP[BAND])
psf_in_t = np.fft.rfft(np.fft.ifftshift(psf_in))
psf_inp = pixelate_psf(psf_in)
psf_inp_t = np.fft.rfft(np.fft.ifftshift(psf_inp))

psf_out = psf_gaussian(SIGMA[BAND] * 2)
psf_out_t = np.fft.rfft(np.fft.ifftshift(psf_out))
weight_t = psf_out_t / psf_inp_t
weight_t[NPIX::] = 0; weight_t.imag = 0
weight = np.fft.ifftshift(np.fft.irfft(weight_t, n=NTOT))

/tmp/ipykernel_11422/375101590.py:11: RuntimeWarning: invalid value encountered in divide
  weight_t = psf_out_t / psf_inp_t


In [3]:
# Figure 1: 1D PSFs used in this work

fig, axs = plt.subplots(3, 1, figsize=(4.8, 5.4))
x = np.arange(NTOT//4) / SAMP
labels = ["Unpixelated Input PSF", "Pixelated Input PSF",
          "Target Output PSF", "Weight Field (Oversampled)"]
styles = ["--", "-", "-.", ":"]

for func, label, ls in zip([psf_in, psf_inp, psf_out, weight * SAMP], labels, styles):
    axs[0].plot(x, func[NTOT//2:NTOT//4*3], ls=ls,
                label=label if "Input" in label else None)
    axs[1].plot(x, (np.cumsum(func[NTOT//2:NTOT//4*3])-func[NTOT//2])/SAMP*2, ls=ls,
                label=label if "Input" not in label else None)
    print(f"{label:27s}: sum={func.sum()/SAMP:.12f}")

axs[0].set_ylabel("PSF or Weight")
axs[0].set_xlabel("$x$ [native pixel]")
axs[1].set_ylabel("Cumulative")
axs[1].set_xlabel("$r$ [native pixel]")
for i in range(2):
    axs[i].legend()

for func, ls in zip([psf_in_t/SAMP, psf_inp_t/SAMP, psf_out_t/SAMP, weight_t], styles):
    axs[2].plot(np.fft.rfftfreq(NTOT, d=1/SAMP)[:NPIX*2], func.real[:NPIX*2], ls=ls)
axs[2].set_ylabel("Fourier Transform")
axs[2].set_xlabel("$u$ [cycle per pixel]")

for ax in axs: ax.grid()
fig.tight_layout()
# plt.show()
fig.savefig(f"plots/psfs_1d.pdf", bbox_inches="tight")
plt.close(fig)

Unpixelated Input PSF      : sum=0.988672515403
Pixelated Input PSF        : sum=0.988672515403
Target Output PSF          : sum=1.000000000000
Weight Field (Oversampled) : sum=1.011457266608


In [4]:
ovl_ii = psf_overlap(psf_inp, psf_inp)  # A matrix.

def explore_width(factor: float = 1.0):
    my_psf_out = psf_gaussian(SIGMA[BAND] * factor)
    my_psf_out_t = np.fft.rfft(np.fft.ifftshift(my_psf_out))
    my_weight_t = my_psf_out_t / psf_inp_t
    my_weight_t[NPIX::] = 0; my_weight_t.imag = 0
    my_weight = np.fft.ifftshift(np.fft.irfft(my_weight_t, n=NTOT))

    my_ovl_oi = psf_overlap(my_psf_out, psf_inp)  # -B/2 matrix.
    my_ovl_oo = psf_overlap(my_psf_out, my_psf_out)  # Center is C.
    C = my_ovl_oo[NTOT//2]
    Amat, Ainv, Bmat, Tmat = explore_case(ovl_ii, my_ovl_oi, shifts=[0])

    my_weightu_i = np.zeros_like(my_weight)
    my_weightu_i[::SAMP] = Tmat[0]
    my_weightu_it = np.fft.rfft(np.fft.ifftshift(my_weightu_i))
    my_psf_outu_i = np.fft.ifftshift(np.fft.irfft(my_weightu_it * psf_inp_t, n=NTOT))

    my_weightu_f = np.zeros_like(my_weight)
    my_weightu_f[::SAMP] = my_weight[::SAMP] * SAMP
    my_weightu_ft = np.fft.rfft(np.fft.ifftshift(my_weightu_f))
    my_psf_outu_f = np.fft.ifftshift(np.fft.irfft(my_weightu_ft * psf_inp_t, n=NTOT))

    return (square_norm(my_psf_outu_i-my_psf_out)/C, square_norm(my_weightu_i),
            square_norm(my_psf_outu_f-my_psf_out)/C, square_norm(my_weightu_f))

In [5]:
# Figure 2: Width of target output PSF

fig, axs = plt.subplots(4, 2, figsize=(9.6, 7.2))
x1 = np.arange(NTOT//4) / SAMP
x2 = np.arange(NTOT//2) / SAMP
d = 0.05

for j, factor in enumerate([1, 2, 3]):
    for i in range(2):
        axs[j, i].set_title(r"Target $\sigma =" + f"{SIGMA[BAND] * factor:.4f}" + "$ [native pixel]")

    my_psf_out = psf_gaussian(SIGMA[BAND] * factor)
    my_psf_out_t = np.fft.rfft(np.fft.ifftshift(my_psf_out))
    my_weight_t = my_psf_out_t / psf_inp_t
    my_weight_t[NPIX::] = 0; my_weight_t.imag = 0
    my_weight = np.fft.ifftshift(np.fft.irfft(my_weight_t, n=NTOT))

    axs[j, 0].plot(x1, my_weight[NTOT//2:NTOT//4*3]*SAMP, c="k", label="Oversampled")
    axs[j, 0].set_ylabel("Weight")
    axs[j, 0].legend()

    my_ovl_oi = psf_overlap(my_psf_out, psf_inp)  # -B/2 matrix.
    my_ovl_oo = psf_overlap(my_psf_out, my_psf_out)  # Center is C.
    C = my_ovl_oo[NTOT//2]
    Amat, Ainv, Bmat, Tmat = explore_case(ovl_ii, my_ovl_oi, shifts=[0])

    for p in range(16):
        axs[j, 0].plot([p-d,]*2, [0, Tmat[0, NPIX//2+p]], c="C0")
        axs[j, 0].plot([p+d,]*2, [0, my_weight[NTOT//2+SAMP*p]*SAMP], c="C1")

    my_weightu_i = np.zeros_like(my_weight)
    my_weightu_i[::SAMP] = Tmat[0]
    my_weightu_it = np.fft.rfft(np.fft.ifftshift(my_weightu_i))
    my_psf_outu_i = np.fft.ifftshift(np.fft.irfft(my_weightu_it * psf_inp_t, n=NTOT))
    axs[j, 1].plot(x2, (my_psf_outu_i-my_psf_out)[NTOT//2:], c="C0",
                   label=f"IMCOM, $U/C$ = {square_norm(my_psf_outu_i-my_psf_out)/C:.4e}")

    my_weightu_f = np.zeros_like(my_weight)
    my_weightu_f[::SAMP] = my_weight[::SAMP] * SAMP
    my_weightu_ft = np.fft.rfft(np.fft.ifftshift(my_weightu_f))
    my_psf_outu_f = np.fft.ifftshift(np.fft.irfft(my_weightu_ft * psf_inp_t, n=NTOT))
    axs[j, 1].plot(x2, (my_psf_outu_f-my_psf_out)[NTOT//2:], c="C1",
                   label=f"Fast IMCOM, $U/C$ = {square_norm(my_psf_outu_f-my_psf_out)/C:.4e}")

    axs[j, 1].set_ylabel("PSF Residual")
    if j == 2:
        axs[j, 1].legend(loc="upper right")
    else:
        h, l = axs[j, 1].get_legend_handles_labels()
        axs[j, 1].add_artist(axs[j, 1].legend(h[:1], l[:1], loc="upper right"))
        axs[j, 1].legend(h[1:], l[1:], loc="lower right")

for i in range(2):
    axs[2, i].set_xlabel("$x$ [native pixel]")

stats = np.zeros((31, 4))
factors = np.linspace(1, 4, 31)
for k, factor in enumerate(factors):
    stats[k] = explore_width(factor)
axs[3, 0].plot(SIGMA[BAND] * factors, stats[:, 0], label="IMCOM")
axs[3, 0].plot(SIGMA[BAND] * factors, stats[:, 2], label="Fast IMCOM")
axs[3, 0].set_ylabel("PSF Leakage $U/C$")
axs[3, 1].plot(SIGMA[BAND] * factors, stats[:, 1], label="IMCOM")
axs[3, 1].plot(SIGMA[BAND] * factors, stats[:, 3], label="Fast IMCOM")
axs[3, 1].set_ylabel(r"Noise Amplification $\Sigma$")

for i in range(2):
    for factor in [1, 2, 3]:
        axs[3, i].axvline(SIGMA[BAND] * factor, ls=":", c="k")

    axs[3, i].set_yscale("log")
    axs[3, i].set_xlabel(r"Target $\sigma$ [native pixel]")
    axs[3, i].legend()

for ax in axs.ravel(): ax.grid()
fig.tight_layout()
# plt.show()
fig.savefig(f"plots/target_width.pdf", bbox_inches="tight")
plt.close(fig)

/tmp/ipykernel_11422/1662956993.py:14: RuntimeWarning: invalid value encountered in divide
  my_weight_t = my_psf_out_t / psf_inp_t
/tmp/ipykernel_11422/1662956993.py:14: RuntimeWarning: divide by zero encountered in divide
  my_weight_t = my_psf_out_t / psf_inp_t
/tmp/ipykernel_11422/562287573.py:6: RuntimeWarning: invalid value encountered in divide
  my_weight_t = my_psf_out_t / psf_inp_t
/tmp/ipykernel_11422/562287573.py:6: RuntimeWarning: divide by zero encountered in divide
  my_weight_t = my_psf_out_t / psf_inp_t


# 4. Image Regridding

In [6]:
# 2025.09.07 coadd_1d.ipynb
ovl_ii = psf_overlap(psf_inp, psf_inp)  # A matrix.
ovl_oi = psf_overlap(psf_out, psf_inp)  # -B/2 matrix.
ovl_oo = psf_overlap(psf_out, psf_out)  # Center is C.
print(C := ovl_oo[NTOT//2], square_norm(psf_out))

4.831145238312438 4.831145238312438


In [7]:
# Figure 3: IMCOM matrices for regridding
Amat, Ainv, Bmat, Tmat = explore_case(ovl_ii, ovl_oi, shifts=[0], aspect=13, figname="imcom_mats1")

In [8]:
def explore_shift(shift: int = 0):
    my_weightu_i = np.zeros_like(weight)
    my_weightu_i[shift::SAMP] = Tmat[shift]
    my_weightu_f = np.zeros_like(weight)
    my_weightu_f[shift::SAMP] = weight[shift::SAMP] * SAMP

    my_psf_outu_it = np.fft.rfft(np.fft.ifftshift(my_weightu_i))
    my_psf_outu_i = np.fft.ifftshift(np.fft.irfft(my_psf_outu_it * psf_inp_t, n=NTOT))
    my_psf_outu_ft = np.fft.rfft(np.fft.ifftshift(my_weightu_f))
    my_psf_outu_f = np.fft.ifftshift(np.fft.irfft(my_psf_outu_ft * psf_inp_t, n=NTOT))

    return (square_norm(my_psf_outu_i-psf_out)/C, square_norm(my_weightu_i),
            square_norm(my_psf_outu_f-psf_out)/C, square_norm(my_weightu_f),
            np.einsum("ji,j,i->", Amat, Tmat[shift], Tmat[shift]) / C\
                - 2 * np.einsum("j,j->", Bmat[shift], Tmat[shift]) / C + 1)

In [9]:
# Figure 4: Relative position of output pixel

fig, axs = plt.subplots(4, 2, figsize=(9.6, 7.2))
x = np.arange(NTOT//4) / SAMP

for j, shift in enumerate([4, 8, 16]):
    for i in range(2):
        axs[j, i].set_title(r"$\Delta x =" + f"{shift/SAMP:.4f}" + "$ [native pixel]")

    my_weightu_i = np.zeros_like(weight)
    my_weightu_i[shift::SAMP] = Tmat[shift]
    my_weightu_f = np.zeros_like(weight)
    my_weightu_f[shift::SAMP] = weight[shift::SAMP] * SAMP
    # print(np.einsum("ji,j,i->", Amat, Tmat[shift], Tmat[shift]) / C\
    #     - 2 * np.einsum("j,j->", Bmat[shift], Tmat[shift]) / C + 1)

    psf_outu_it = np.fft.rfft(np.fft.ifftshift(my_weightu_i))
    psf_outu_i = np.fft.ifftshift(np.fft.irfft(psf_outu_it * psf_inp_t, n=NTOT))
    psf_outu_ft = np.fft.rfft(np.fft.ifftshift(my_weightu_f))
    psf_outu_f = np.fft.ifftshift(np.fft.irfft(psf_outu_ft * psf_inp_t, n=NTOT))

    for psf_outu, label, c in [(psf_outu_i, "IMCOM", "C0"),
                               (psf_outu_f, "Fast IMCOM", "C1")]:
        axs[j, 0].plot(x, (psf_outu-psf_out)[NTOT//2:NTOT//4*3], c=c,
                       label=f"{label}, $U/C$ = {square_norm(psf_outu-psf_out)/C:.4e}")
        for part, ls in zip(["real", "imag"], ["-", "--"]):
            axs[j, 1].plot(np.fft.rfftfreq(NTOT, d=1/SAMP)[:1*NPIX], getattr(
                np.fft.rfft(np.fft.ifftshift(psf_outu-psf_out)), part)[:1*NPIX], ls=ls, c=c)

    h, l = axs[j, 0].get_legend_handles_labels()
    axs[j, 0].add_artist(axs[j, 0].legend(h[:1], l[:1], loc="upper right"))
    axs[j, 0].legend(h[1:], l[1:], loc="lower right")
    axs[j, 0].set_ylabel("PSF Residual")
    axs[j, 1].set_ylabel("Fourier Transform")

ax = axs[1, 1]
ax.plot([], [], "k-", label="Real part")
ax.plot([], [], "k--", label="Imaginary part")
ax.legend(loc="lower left")

axs[2, 0].set_xlabel("$x$ [native pixel]")
axs[2, 1].set_xlabel("$u$ [cycle per pixel]")

stats = np.zeros((SAMP, 5))
shifts = np.arange(SAMP)
for k, shift in enumerate(shifts):
    stats[k] = explore_shift(shift)
axs[3, 0].plot(shifts / SAMP, stats[:, 0], label="IMCOM (actual)")
axs[3, 0].plot(shifts / SAMP, stats[:, 2], label="Fast IMCOM")
axs[3, 0].plot(shifts / SAMP, stats[:, 4], c="C3", ls="--", label="IMCOM (reported)")
axs[3, 0].set_ylabel("PSF Leakage $U/C$")
axs[3, 1].plot(shifts / SAMP, stats[:, 1], label="IMCOM")
axs[3, 1].plot(shifts / SAMP, stats[:, 3], label="Fast IMCOM")
axs[3, 1].set_ylabel(r"Noise Amplification $\Sigma$")

for i in range(2):
    axs[3, i].set_xlabel(r"$\Delta x$ [native pixel]")
    axs[3, i].legend(loc="center")

for ax in axs.ravel(): ax.grid()
fig.tight_layout()
# plt.show()
fig.savefig(f"plots/sub_shifts1.pdf", bbox_inches="tight")
plt.close(fig)

# 5. Image Coaddition

In [10]:
# Figure 6: IMCOM matrices for coaddition
Amat, Ainv, Bmat, Tmat = explore_case(ovl_ii, ovl_oi, shifts=[0, 16], aspect=5, figname="imcom_mats2")

In [11]:
def explore_shifts2(shifts: list[int] = [0, 16]):
    Amat, Ainv, Bmat, Tmat = explore_case(ovl_ii, ovl_oi, shifts=shifts)

    my_weightu_i = np.zeros_like(weight); sigma_i = 0.0
    my_weightu_f = np.zeros_like(weight); sigma_f = 0.0
    for i, s in enumerate(shifts):
        my_weightu_i[s::SAMP] += Tmat[0, i*NPIX:(i+1)*NPIX]
        sigma_i += square_norm(Tmat[0, i*NPIX:(i+1)*NPIX])
        my_weightu_f[s::SAMP] += weight[s::SAMP] * SAMP / 2
        sigma_f += square_norm(weight[s::SAMP] * SAMP / 2)

    my_psf_outu_it = np.fft.rfft(np.fft.ifftshift(my_weightu_i))
    my_psf_outu_i = np.fft.ifftshift(np.fft.irfft(my_psf_outu_it * psf_inp_t, n=NTOT))
    my_psf_outu_ft = np.fft.rfft(np.fft.ifftshift(my_weightu_f))
    my_psf_outu_f = np.fft.ifftshift(np.fft.irfft(my_psf_outu_ft * psf_inp_t, n=NTOT))

    return (square_norm(my_psf_outu_i-psf_out)/C, sigma_i,
            square_norm(my_psf_outu_f-psf_out)/C, sigma_f,
            np.einsum("ji,j,i->", Amat, Tmat[0], Tmat[0]) / C\
                - 2 * np.einsum("j,j->", Bmat[0], Tmat[0]) / C + 1)

In [12]:
# Figure 7: Coaddition of 2 images

fig, axs = plt.subplots(4, 2, figsize=(9.6, 7.2))
x = np.arange(NTOT//4) / SAMP

for j, shift in enumerate([4, 8, 16]):
    for i in range(2):
        axs[j, i].set_title(r"$\Delta x_0 = 0$, $\Delta x_1 =" +\
            f"{shift/SAMP:.4f}" + "$ [native pixel]")

    shifts = [0, shift]  # Shifts of the input grids.
    Amat, Ainv, Bmat, Tmat = explore_case(ovl_ii, ovl_oi, shifts=shifts)

    my_weightu_i = np.zeros_like(weight)
    my_weightu_f = np.zeros_like(weight)
    for i, s in enumerate(shifts):
        my_weightu_i[s::SAMP] += Tmat[0, i*NPIX:(i+1)*NPIX]
        my_weightu_f[s::SAMP] += weight[s::SAMP] * SAMP / 2
    # print(np.einsum("ji,j,i->", Amat, Tmat[0], Tmat[0]) / C\
    #     - 2 * np.einsum("j,j->", Bmat[0], Tmat[0]) / C + 1)

    psf_outu_it = np.fft.rfft(np.fft.ifftshift(my_weightu_i))
    psf_outu_i = np.fft.ifftshift(np.fft.irfft(psf_outu_it * psf_inp_t, n=NTOT))
    psf_outu_ft = np.fft.rfft(np.fft.ifftshift(my_weightu_f))
    psf_outu_f = np.fft.ifftshift(np.fft.irfft(psf_outu_ft * psf_inp_t, n=NTOT))

    for psf_outu, label, c in [(psf_outu_i, "IMCOM", "C0"),
                               (psf_outu_f, "Fast IMCOM", "C1")]:
        axs[j, 0].plot(x, (psf_outu-psf_out)[NTOT//2:NTOT//4*3], c=c,
                       label=f"{label}, $U/C$ = {square_norm(psf_outu-psf_out)/C:.4e}")
        for part, ls in zip(["real", "imag"], ["-", "--"]):
            axs[j, 1].plot(np.fft.rfftfreq(NTOT, d=1/SAMP)[:1*NPIX], getattr(
                np.fft.rfft(np.fft.ifftshift(psf_outu-psf_out)), part)[:1*NPIX], ls=ls, c=c)

    h, l = axs[j, 0].get_legend_handles_labels()
    axs[j, 0].add_artist(axs[j, 0].legend(h[:1], l[:1], loc="upper right"))
    axs[j, 0].legend(h[1:], l[1:], loc="lower right")
    axs[j, 0].set_ylabel("PSF Residual")
    axs[j, 1].set_ylabel("Fourier Transform")

ax = axs[0, 1]
ax.plot([], [], "k-", label="Real part")
ax.plot([], [], "k--", label="Imaginary part")
ax.legend(loc="upper left")

axs[2, 0].set_xlabel("$x$ [native pixel]")
axs[2, 1].set_xlabel("$u$ [cycle per pixel]")

stats = np.zeros((SAMP, 5))
shifts = np.arange(SAMP)
for k, shift in enumerate(shifts):
    stats[k] = explore_shifts2([0, shift])
axs[3, 0].plot(shifts / SAMP, stats[:, 0], label="IMCOM (actual)")
axs[3, 0].plot(shifts / SAMP, stats[:, 2], label="Fast IMCOM")
axs[3, 0].plot(shifts / SAMP, stats[:, 4], c="C3", ls="--", label="IMCOM (reported)")
axs[3, 0].set_ylabel("PSF Leakage $U/C$")
axs[3, 1].plot(shifts / SAMP, stats[:, 1], label="IMCOM")
axs[3, 1].plot(shifts / SAMP, stats[:, 3], label="Fast IMCOM")
axs[3, 1].set_ylabel(r"Noise Amplification $\Sigma$")

for i in range(2):
    axs[3, i].set_yscale("symlog", linthresh=1e-6)
    axs[3, i].set_xlabel(r"$\Delta x_1$ [native pixel]")
h, l = axs[3, 0].get_legend_handles_labels()
axs[3, 0].add_artist(axs[3, 0].legend(h[:1], l[:1], loc="upper right"))
axs[3, 0].legend(h[1:], l[1:], ncol=2, loc="lower right")
axs[3, 1].legend(loc="upper right")

for ax in axs.ravel(): ax.grid()
fig.tight_layout()
# plt.show()
fig.savefig(f"plots/sub_shifts2.pdf", bbox_inches="tight")
plt.close(fig)

In [13]:
def explore_shifts3(shifts: list[int] = [0, 11, 22]):
    meta_weights = get_meta_weights(shifts)
    Amat, Ainv, Bmat, Tmat = explore_case(ovl_ii, ovl_oi, shifts=shifts)

    my_weightu_i = np.zeros_like(weight); sigma_i = 0.0
    my_weightu_fu = np.zeros_like(weight); sigma_fu = 0.0  # PSF leakage first.
    my_weightu_fs = np.zeros_like(weight); sigma_fs = 0.0  # Noise amplification first.
    for i, s in enumerate(shifts):
        my_weightu_i[s::SAMP] += Tmat[0, i*NPIX:(i+1)*NPIX]
        sigma_i += square_norm(Tmat[0, i*NPIX:(i+1)*NPIX])
        my_weightu_fu[s::SAMP] += weight[s::SAMP] * SAMP * meta_weights[i]
        sigma_fu += square_norm(weight[s::SAMP] * SAMP * meta_weights[i])
        my_weightu_fs[s::SAMP] += weight[s::SAMP] * SAMP / 3
        sigma_fs += square_norm(weight[s::SAMP] * SAMP / 3)

    my_psf_outu_it = np.fft.rfft(np.fft.ifftshift(my_weightu_i))
    my_psf_outu_i = np.fft.ifftshift(np.fft.irfft(my_psf_outu_it * psf_inp_t, n=NTOT))
    my_psf_outu_fut = np.fft.rfft(np.fft.ifftshift(my_weightu_fu))
    my_psf_outu_fu = np.fft.ifftshift(np.fft.irfft(my_psf_outu_fut * psf_inp_t, n=NTOT))
    my_psf_outu_fst = np.fft.rfft(np.fft.ifftshift(my_weightu_fs))
    my_psf_outu_fs = np.fft.ifftshift(np.fft.irfft(my_psf_outu_fst * psf_inp_t, n=NTOT))

    return (square_norm(my_psf_outu_i-psf_out)/C, sigma_i,
            square_norm(my_psf_outu_fu-psf_out)/C, sigma_fu,
            square_norm(my_psf_outu_fs-psf_out)/C, sigma_fs,
            np.einsum("ji,j,i->", Amat, Tmat[0], Tmat[0]) / C\
                - 2 * np.einsum("j,j->", Bmat[0], Tmat[0]) / C + 1)

In [14]:
# Figure 8: Coaddition of 3 images

fig, axs = plt.subplots(4, 2, figsize=(9.6, 7.2))
x = np.arange(NTOT//4) / SAMP

for j, shift in enumerate([4, 8, 12]):
    for i in range(2):
        axs[j, i].set_title(r"$\Delta x_0 = 0$, $\Delta x_1 = \Delta x_2 / 2 =" +\
            f"{shift/SAMP:.4f}" + "$ [native pixel]")

    shifts = [0, shift, shift*2]  # Shifts of the input grids.
    meta_weights = get_meta_weights(shifts)
    Amat, Ainv, Bmat, Tmat = explore_case(ovl_ii, ovl_oi, shifts=shifts)

    my_weightu_i = np.zeros_like(weight)
    my_weightu_fu = np.zeros_like(weight)  # PSF leakage first.
    my_weightu_fs = np.zeros_like(weight)  # Noise amplification first.
    for i, s in enumerate(shifts):
        my_weightu_i[s::SAMP] += Tmat[0, i*NPIX:(i+1)*NPIX]
        my_weightu_fu[s::SAMP] += weight[s::SAMP] * SAMP * meta_weights[i]
        my_weightu_fs[s::SAMP] += weight[s::SAMP] * SAMP / 3
    # print(np.einsum("ji,j,i->", Amat, Tmat[0], Tmat[0]) / C\
    #     - 2 * np.einsum("j,j->", Bmat[0], Tmat[0]) / C + 1)

    psf_outu_it = np.fft.rfft(np.fft.ifftshift(my_weightu_i))
    psf_outu_i = np.fft.ifftshift(np.fft.irfft(psf_outu_it * psf_inp_t, n=NTOT))
    psf_outu_fut = np.fft.rfft(np.fft.ifftshift(my_weightu_fu))
    psf_outu_fu = np.fft.ifftshift(np.fft.irfft(psf_outu_fut * psf_inp_t, n=NTOT))
    psf_outu_fst = np.fft.rfft(np.fft.ifftshift(my_weightu_fs))
    psf_outu_fs = np.fft.ifftshift(np.fft.irfft(psf_outu_fst * psf_inp_t, n=NTOT))

    for psf_outu, label, c in [(psf_outu_i, "IMCOM", "C0"),
                               (psf_outu_fu, "Fast IMCOM ($U$-first)", "C1"),
                               (psf_outu_fs, r"Fast IMCOM ($\Sigma$-first)", "C2")]:
        axs[j, 0].plot(x, (psf_outu-psf_out)[NTOT//2:NTOT//4*3], c=c,
                       label=f"{label}, $U/C$ = {square_norm(psf_outu-psf_out)/C:.4e}")
        for part, ls in zip(["real", "imag"], ["-", "--"]):
            axs[j, 1].plot(np.fft.rfftfreq(NTOT, d=1/SAMP)[:1*NPIX], getattr(
                np.fft.rfft(np.fft.ifftshift(psf_outu-psf_out)), part)[:1*NPIX], ls=ls, c=c)

    h, l = axs[j, 0].get_legend_handles_labels()
    axs[j, 0].add_artist(axs[j, 0].legend(h[:1], l[:1], loc="upper right"))
    axs[j, 0].legend(h[1:], l[1:], loc="lower right")
    axs[j, 0].set_ylabel("PSF Residual")
    axs[j, 1].set_ylabel("Fourier Transform")

ax = axs[1, 1]
ax.plot([], [], "k-", label="Real part")
ax.plot([], [], "k--", label="Imaginary part")
ax.legend(loc="upper left")

axs[2, 0].set_xlabel("$x$ [native pixel]")
axs[2, 1].set_xlabel("$u$ [cycle per pixel]")

stats = np.zeros((SAMP//2-1, 7))
shifts = np.arange(1, SAMP//2)
for k, shift in enumerate(shifts):
    stats[k] = explore_shifts3([0, shift, shift*2])
axs[3, 0].plot(shifts / SAMP, stats[:, 0], label="IMCOM (actual)")
axs[3, 0].plot(shifts / SAMP, stats[:, 2], label="Fast IMCOM ($U$-first)")
axs[3, 0].plot(shifts / SAMP, stats[:, 4], label="Fast IMCOM ($\Sigma$-first)")
axs[3, 0].plot(shifts / SAMP, stats[:, 6], c="C3", ls="--", label="IMCOM (reported)")
axs[3, 0].set_ylabel("PSF Leakage $U/C$")
axs[3, 1].plot(shifts / SAMP, stats[:, 1], label="IMCOM")
axs[3, 1].plot(shifts / SAMP, stats[:, 3], label="Fast IMCOM ($U$-first)")
axs[3, 1].plot(shifts / SAMP, stats[:, 5], label="Fast IMCOM ($\Sigma$-first)")
axs[3, 1].set_ylabel(r"Noise Amplification $\Sigma$")

for i in range(2):
    axs[3, i].set_yscale("symlog", linthresh=1e-6)
    axs[3, i].set_xlabel(r"$\Delta x_1 = \Delta x_2 / 2$ [native pixel]")
h, l = axs[3, 0].get_legend_handles_labels()
axs[3, 0].add_artist(axs[3, 0].legend(h[:2], l[:2], ncol=2, loc="upper right"))
axs[3, 0].legend(h[2:], l[2:], ncol=2, loc="lower right")
axs[3, 1].legend(loc="upper right")

for ax in axs.ravel(): ax.grid()
fig.tight_layout()
# plt.show()
fig.savefig(f"plots/sub_shifts3.pdf", bbox_inches="tight")
plt.close(fig)

In [15]:
# Figure 9: Balance between U/C and Sigma

stats = np.zeros((31, 31, 7))
for j, shift1 in enumerate(range(1, SAMP)):
    for i, shift2 in enumerate(range(1, SAMP)):
        stats[j, i] = explore_shifts3([0, shift1, shift2])

In [16]:
fig, axs = plt.subplots(2, 3, figsize=(9.6, 5.4), sharex=True, sharey=True)
extent = [0.5/SAMP, (SAMP-0.5)/SAMP, 0.5/SAMP, (SAMP-0.5)/SAMP]

for j, map_ in enumerate(["$U/C$", "$\Sigma$"]):
    for i, method in enumerate(["IMCOM", "Fast IMCOM ($U$-first)",
                                "Fast IMCOM ($\Sigma$-first)"]):
        ax = axs[j, i]
        if (j, i) != (1, 2):
            im = ax.imshow(stats[:, :, i*2+j], origin="lower", extent=extent,
                           norm=LogNorm(), cmap=["inferno_r", "magma_r"][j])
        else:
            median = np.median(stats[:, :, i*2+j])
            im = ax.imshow(stats[:, :, i*2+j], vmin=median-1e-2, vmax=median+1e-2,
                           origin="lower", extent=extent, norm=None, cmap="magma_r")
        fig.colorbar(im, ax=ax)
        ax.set_title(f"{method}: {map_}")
        ax.scatter([1/3, 2/3], [2/3, 1/3], c="b", marker="+", s=25)

for j in range(2):
    axs[j, 0].set_ylabel(r"$\Delta x_2$ [native pixel]")
for i in range(3):
    axs[1, i].set_xlabel(r"$\Delta x_1$ [native pixel]")

fig.tight_layout()
# plt.show()
fig.savefig(f"plots/usigma_trade.pdf", bbox_inches="tight")
plt.close(fig)

In [17]:
median

0.08511300546993615

# A. Impact of Asymmetric Windows

In [18]:
def explore_reject1(reject: int = 0):
    Amat, Ainv, Bmat, Tmat = explore_case(
        ovl_ii, ovl_oi, shifts=[0], reject=reject)

    my_weightu_i = np.zeros_like(weight)
    my_weightu_i[reject*SAMP::SAMP] = Tmat[0]
    my_weightu_f = np.zeros_like(weight)
    my_weightu_f[reject*SAMP::SAMP] = weight[reject*SAMP::SAMP] * SAMP

    my_psf_outu_it = np.fft.rfft(np.fft.ifftshift(my_weightu_i))
    my_psf_outu_i = np.fft.ifftshift(np.fft.irfft(my_psf_outu_it * psf_inp_t, n=NTOT))
    my_psf_outu_ft = np.fft.rfft(np.fft.ifftshift(my_weightu_f))
    my_psf_outu_f = np.fft.ifftshift(np.fft.irfft(my_psf_outu_ft * psf_inp_t, n=NTOT))

    return (square_norm(my_psf_outu_i-psf_out)/C, square_norm(my_weightu_i),
            square_norm(my_psf_outu_f-psf_out)/C, square_norm(my_weightu_f),
            np.einsum("ji,j,i->", Amat, Tmat[0], Tmat[0]) / C\
                - 2 * np.einsum("j,j->", Bmat[0], Tmat[0]) / C + 1)

In [19]:
# Figure 10: Impact on regridding

fig, axs = plt.subplots(4, 2, figsize=(9.6, 7.2))
x = np.arange(-NTOT//2, NTOT//4) / SAMP

for j, reject in enumerate([8, 16, 24]):
    for i in range(2):
        axs[j, i].set_title(f"${reject}$ lost pixels on the left")
    axs[j, 0].axvline(-32.5 + reject, ls=":", c="k")

    Amat, Ainv, Bmat, Tmat = explore_case(
        ovl_ii, ovl_oi, shifts=[0], reject=reject)
    # print(np.einsum("ji,j,i->", Amat, Tmat[0], Tmat[0]) / C\
    #     - 2 * np.einsum("j,j->", Bmat[0], Tmat[0]) / C + 1)

    my_weightu_i = np.zeros_like(weight)
    my_weightu_i[reject*SAMP::SAMP] = Tmat[0]
    my_weightu_f = np.zeros_like(weight)
    my_weightu_f[reject*SAMP::SAMP] = weight[reject*SAMP::SAMP] * SAMP

    psf_outu_it = np.fft.rfft(np.fft.ifftshift(my_weightu_i))
    psf_outu_i = np.fft.ifftshift(np.fft.irfft(psf_outu_it * psf_inp_t, n=NTOT))
    psf_outu_ft = np.fft.rfft(np.fft.ifftshift(my_weightu_f))
    psf_outu_f = np.fft.ifftshift(np.fft.irfft(psf_outu_ft * psf_inp_t, n=NTOT))

    for psf_outu, label, c in [(psf_outu_i, "IMCOM", "C0"),
                               (psf_outu_f, "Fast IMCOM", "C1")]:
        axs[j, 0].plot(x, (psf_outu-psf_out)[:NTOT//4*3], c=c,
                       label=f"{label}, $U/C$ = {square_norm(psf_outu-psf_out)/C:.4e}")
        for part, ls in zip(["real", "imag"], ["-", "--"]):
            axs[j, 1].plot(np.fft.rfftfreq(NTOT, d=1/SAMP)[:1*NPIX], getattr(
                np.fft.rfft(np.fft.ifftshift(psf_outu-psf_out)), part)[:1*NPIX], ls=ls, c=c)

    axs[j, 0].legend(loc="lower left" if j < 2 else "upper right")
    axs[j, 0].set_ylabel("PSF Residual")
    axs[j, 1].set_ylabel("Fourier Transform")

ax = axs[0, 1]
ax.plot([], [], "k-", label="Real part")
ax.plot([], [], "k--", label="Imaginary part")
ax.legend(loc="upper left")

axs[2, 0].set_xlabel("$x$ [native pixel]")
axs[2, 1].set_xlabel("$u$ [cycle per pixel]")

stats = np.zeros((NPIX//2-4, 5))
rejects = np.arange(NPIX//2-4)
for k, reject in enumerate(rejects):
    stats[k] = explore_reject1(reject)
axs[3, 0].plot(rejects, stats[:, 0], label="IMCOM (actual)")
axs[3, 0].plot(rejects, stats[:, 2], label="Fast IMCOM")
axs[3, 0].plot(rejects, stats[:, 4], c="C3", ls="--", label="IMCOM (reported)")
axs[3, 0].set_ylabel("PSF Leakage $U/C$")
axs[3, 1].plot(rejects, stats[:, 1], label="IMCOM")
axs[3, 1].plot(rejects, stats[:, 3], label="Fast IMCOM")
axs[3, 1].set_ylabel(r"Noise Amplification $\Sigma$")

axs[3, 0].set_yscale("log")
for i in range(2):
    for reject in [8, 16, 24]:
        axs[3, i].axvline(reject, ls=":", c="k")
    axs[3, i].set_xlabel("Number of lost pixels")
axs[3, 0].legend(loc="upper left")
axs[3, 1].legend(loc="lower left")

for ax in axs.ravel(): ax.grid()
fig.tight_layout()
# plt.show()
fig.savefig(f"plots/lost_pixels1.pdf", bbox_inches="tight")
plt.close(fig)

In [20]:
shifts = [0, 8, 20]  # Shifts of the input grids.
meta_weights = get_meta_weights(shifts)

def explore_reject3(reject: int = 0):
    npix = NPIX - reject
    Amat, Ainv, Bmat, Tmat = explore_case(
        ovl_ii, ovl_oi, shifts=shifts, reject=reject)

    my_weightu_i = np.zeros_like(weight); sigma_i = 0.0
    my_weightu_fu = np.zeros_like(weight); sigma_fu = 0.0  # PSF leakage first.
    my_weightu_fs = np.zeros_like(weight); sigma_fs = 0.0  # Noise amplification first.
    for i, s in enumerate(shifts):
        my_weightu_i[reject*SAMP+s::SAMP] = Tmat[0, i*npix:(i+1)*npix]
        sigma_i += square_norm(Tmat[0, i*npix:(i+1)*npix])
        my_weightu_fu[reject*SAMP+s::SAMP] = weight[reject*SAMP+s::SAMP] * SAMP * meta_weights[i]
        sigma_fu += square_norm(weight[reject*SAMP+s::SAMP] * SAMP * meta_weights[i])
        my_weightu_fs[reject*SAMP+s::SAMP] = weight[reject*SAMP+s::SAMP] * SAMP / 3
        sigma_fs += square_norm(weight[reject*SAMP+s::SAMP] * SAMP / 3)

    my_psf_outu_it = np.fft.rfft(np.fft.ifftshift(my_weightu_i))
    my_psf_outu_i = np.fft.ifftshift(np.fft.irfft(my_psf_outu_it * psf_inp_t, n=NTOT))
    my_psf_outu_fut = np.fft.rfft(np.fft.ifftshift(my_weightu_fu))
    my_psf_outu_fu = np.fft.ifftshift(np.fft.irfft(my_psf_outu_fut * psf_inp_t, n=NTOT))
    my_psf_outu_fst = np.fft.rfft(np.fft.ifftshift(my_weightu_fs))
    my_psf_outu_fs = np.fft.ifftshift(np.fft.irfft(my_psf_outu_fst * psf_inp_t, n=NTOT))

    return (square_norm(my_psf_outu_i-psf_out)/C, sigma_i,
            square_norm(my_psf_outu_fu-psf_out)/C, sigma_fu,
            square_norm(my_psf_outu_fs-psf_out)/C, sigma_fs,
            np.einsum("ji,j,i->", Amat, Tmat[0], Tmat[0]) / C\
                - 2 * np.einsum("j,j->", Bmat[0], Tmat[0]) / C + 1)

In [21]:
# Figure 11: Impact on coaddition

fig, axs = plt.subplots(4, 2, figsize=(9.6, 7.2))
x = np.arange(-NTOT//2, NTOT//4) / SAMP

for j, reject in enumerate([8, 16, 24]):
    for i in range(2):
        axs[j, i].set_title(rf"${reject} \times 3$ lost pixels on the left")
    axs[j, 0].axvline(-32.5 + reject, ls=":", c="k")
    npix = NPIX - reject

    Amat, Ainv, Bmat, Tmat = explore_case(
        ovl_ii, ovl_oi, shifts=shifts, reject=reject)
    # print(np.einsum("ji,j,i->", Amat, Tmat[0], Tmat[0]) / C\
    #     - 2 * np.einsum("j,j->", Bmat[0], Tmat[0]) / C + 1)

    my_weightu_i = np.zeros_like(weight)
    my_weightu_fu = np.zeros_like(weight)  # PSF leakage first.
    my_weightu_fs = np.zeros_like(weight)  # Noise amplification first.
    for i, s in enumerate(shifts):
        my_weightu_i[reject*SAMP+s::SAMP] = Tmat[0, i*npix:(i+1)*npix]
        my_weightu_fu[reject*SAMP+s::SAMP] = weight[reject*SAMP+s::SAMP] * SAMP * meta_weights[i]
        my_weightu_fs[reject*SAMP+s::SAMP] = weight[reject*SAMP+s::SAMP] * SAMP / 3

    psf_outu_it = np.fft.rfft(np.fft.ifftshift(my_weightu_i))
    psf_outu_i = np.fft.ifftshift(np.fft.irfft(psf_outu_it * psf_inp_t, n=NTOT))
    psf_outu_fut = np.fft.rfft(np.fft.ifftshift(my_weightu_fu))
    psf_outu_fu = np.fft.ifftshift(np.fft.irfft(psf_outu_fut * psf_inp_t, n=NTOT))
    psf_outu_fst = np.fft.rfft(np.fft.ifftshift(my_weightu_fs))
    psf_outu_fs = np.fft.ifftshift(np.fft.irfft(psf_outu_fst * psf_inp_t, n=NTOT))

    for psf_outu, label, c in [(psf_outu_i, "IMCOM", "C0"),
                               (psf_outu_fu, "Fast IMCOM ($U$-first)", "C1"),
                               (psf_outu_fs, r"Fast IMCOM ($\Sigma$-first)", "C2")]:
        axs[j, 0].plot(x, (psf_outu-psf_out)[:NTOT//4*3], c=c,
                       label=f"{label}, $U/C$ = {square_norm(psf_outu-psf_out)/C:.4e}")
        for part, ls in zip(["real", "imag"], ["-", "--"]):
            axs[j, 1].plot(np.fft.rfftfreq(NTOT, d=1/SAMP)[:1*NPIX], getattr(
                np.fft.rfft(np.fft.ifftshift(psf_outu-psf_out)), part)[:1*NPIX], ls=ls, c=c)

    h, l = axs[j, 0].get_legend_handles_labels()
    axs[j, 0].add_artist(axs[j, 0].legend(h[:1], l[:1], loc="lower left"))
    axs[j, 0].legend(h[1:], l[1:], loc="upper right")
    axs[j, 0].set_ylabel("PSF Residual")
    axs[j, 1].set_ylabel("Fourier Transform")

ax = axs[1, 1]
ax.plot([], [], "k-", label="Real part")
ax.plot([], [], "k--", label="Imaginary part")
ax.legend(loc="upper right")

axs[2, 0].set_xlabel("$x$ [native pixel]")
axs[2, 1].set_xlabel("$u$ [cycle per pixel]")

stats = np.zeros((NPIX//2-6, 7))
rejects = np.arange(NPIX//2-6)
for k, reject in enumerate(rejects):
    stats[k] = explore_reject3(reject)
axs[3, 0].plot(rejects, stats[:, 0], label="IMCOM (actual)")
axs[3, 0].plot(rejects, stats[:, 2], label="Fast IMCOM ($U$-first)")
axs[3, 0].plot(rejects, stats[:, 4], label="Fast IMCOM ($\Sigma$-first)")
axs[3, 0].plot(rejects, stats[:, 6], c="C3", ls="--", label="IMCOM (reported)")
axs[3, 0].set_ylabel("PSF Leakage $U/C$")
axs[3, 1].plot(rejects, stats[:, 1], label="IMCOM")
axs[3, 1].plot(rejects, stats[:, 3], label="Fast IMCOM ($U$-first)")
axs[3, 1].plot(rejects, stats[:, 5], label="Fast IMCOM ($\Sigma$-first)")
axs[3, 1].set_ylabel(r"Noise Amplification $\Sigma$")

for i in range(2):
    for reject in [8, 16, 24]:
        axs[3, i].axvline(reject, ls=":", c="k")
    axs[3, i].set_yscale("log")
    axs[3, i].set_xlabel("Number of lost pixels per image")
axs[3, 0].legend(loc="lower right")
axs[3, 1].legend(loc="center right")

for ax in axs.ravel(): ax.grid()
fig.tight_layout()
# plt.show()
fig.savefig(f"plots/lost_pixels3.pdf", bbox_inches="tight")
plt.close(fig)